In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
player_game_data = pd.read_csv('./player_game_data.csv')
team_game_data = pd.read_csv('./team_game_data.csv')
contract_data = pd.read_csv('./nba_spotrac_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: './player_game_data.csv'

In [5]:
contract_data['cap_hit'] = pd.to_numeric(
    contract_data['cap_hit'].replace({'\$': '', ',': '', '-': None}, regex=True),
    errors='coerce'
)

In [13]:
aggregated_stats = player_game_data.groupby('player_name').agg({
    'points': ['sum', 'mean'],  # Total and Average Points
    'reboffensive': ['sum', 'mean'],  # Total and Average Offensive Rebounds
    'rebdefensive': ['sum', 'mean'],  # Total and Average Defensive Rebounds
    'assists': ['sum', 'mean'],  # Total and Average Assists
    'turnovers': ['sum', 'mean']  # Total and Average Turnovers
}).reset_index()

In [14]:
# Flatten MultiIndex Columns
aggregated_stats.columns = ['_'.join(col).strip('_') for col in aggregated_stats.columns]

In [15]:
# Rename columns for clarity
aggregated_stats.rename(columns={
    'player_': 'player',
    'points_sum': 'total_points',
    'points_mean': 'avg_points',
    'reboffensive_sum': 'total_offensive_rebounds',
    'reboffensive_mean': 'avg_offensive_rebounds',
    'rebdefensive_sum': 'total_defensive_rebounds',
    'rebdefensive_mean': 'avg_defensive_rebounds',
    'assists_sum': 'total_assists',
    'assists_mean': 'avg_assists',
    'turnovers_sum': 'total_turnovers',
    'turnovers_mean': 'avg_turnovers'
}, inplace=True)

In [21]:
contract_data.rename(columns={'player': 'player_name'}, inplace=True)

In [73]:
# 1. Filter data for a specific position (e.g., Guards)
positional_data = contract_data[contract_data['pos'] == 'SG']


In [74]:
player_contract_data = pd.merge(positional_data, aggregated_stats, on='player_name', how='left')

In [75]:
# 3. Feature Engineering (with offensive/defensive rebound split)
player_contract_data['total_rebounds'] = player_contract_data['total_offensive_rebounds'] + player_contract_data['total_defensive_rebounds']
player_contract_data['offensive_to_defensive_rebound_ratio'] = player_contract_data['total_offensive_rebounds'] / (player_contract_data['total_defensive_rebounds'] + 1)  # To avoid division by zero


In [76]:
# Select Features and Target
features = ['total_points', 'avg_points', 'total_rebounds', 'avg_offensive_rebounds', 'avg_defensive_rebounds', 
            'offensive_to_defensive_rebound_ratio', 'total_assists', 'avg_assists', 'total_turnovers']
target = 'cap_hit'

In [77]:
X = player_contract_data[features]
y = player_contract_data[target]

In [78]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline,Pipeline

# Initialize the SimpleImputer for numerical columns (mean imputation by default)
X_imputer = SimpleImputer(strategy='mean')  # For imputing missing values in X
y_imputer = SimpleImputer(strategy='mean')

# Apply imputation to X and y
X_imputed = X_imputer.fit_transform(X)
y_imputed = y_imputer.fit_transform(y.values.reshape(-1, 1)).ravel() 

In [79]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_imputed, test_size=0.2, random_state=42)


In [81]:
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor(random_state=42)


In [82]:
pipeline = make_pipeline(SimpleImputer(strategy='mean'), model)

In [83]:
param_grid = {
    'histgradientboostingregressor__max_iter': [100, 200, 300],
    'histgradientboostingregressor__learning_rate': [0.01, 0.05, 0.1],
    'histgradientboostingregressor__max_depth': [None, 10, 20],
    'histgradientboostingregressor__min_samples_leaf': [20, 50, 100],
    'histgradientboostingregressor__l2_regularization': [0, 0.1, 0.2]
}

In [84]:
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

/Users/mugilant/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/mugilant/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/mugilant/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/Users/mugilant/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/mugilant/Library/Python/3.9/lib/python/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/Users/mugilant/Library/Python/3.9/lib/python/site-packages/sklear

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('simpleimputer', SimpleImputer()),
                                       ('histgradientboostingregressor',
                                        HistGradientBoostingRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'histgradientboostingregressor__l2_regularization': [0,
                                                                              0.1,
                                                                              0.2],
                         'histgradientboostingregressor__learning_rate': [0.01,
                                                                          0.05,
                                                                          0.1],
                         'histgradientboostingregressor__max_depth': [None, 10,
                                                                      20],
                         'histgradientboostingregressor__max_iter': [100, 200,
                                                                     300],
                         'histgradientboostingregressor__min_samples_leaf': [20,
                                                                             50,
                                                                             100]},
             scoring='accuracy')

In [85]:
best_model = grid_search.best_estimator_

In [86]:
y_pred = best_model.predict(X_test)

In [87]:
print("Best Parameters:", grid_search.best_params_)
print("R² Score:", r2_score(y_test, y_pred))
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", np.sqrt(mean_squared_error(y_test, y_pred)))


Best Parameters: {'histgradientboostingregressor__l2_regularization': 0, 'histgradientboostingregressor__learning_rate': 0.01, 'histgradientboostingregressor__max_depth': None, 'histgradientboostingregressor__max_iter': 100, 'histgradientboostingregressor__min_samples_leaf': 20}
R² Score: 0.3781411115686908
Mean Absolute Error (MAE): 4510876.6084418455
Root Mean Squared Error (RMSE): 6406914.8155586375
